In [2]:
!pip install -qU  langchain-core==0.2.9
!pip install -qU langchain-community==0.2.5
!pip install -qU  crewai[tools]==0.32.0
!pip install -qU langchain-groq==0.1.5

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.2.11 requires langchain-core<0.3.0,>=0.2.23, but you have langchain-core 0.2.9 which is incompatible.
langchain-text-splitters 0.2.2 requires langchain-core<0.3.0,>=0.2.10, but you have langchain-core 0.2.9 which is incompatible.


In [5]:
import pandas as pd

In [16]:
import json
import os
import sqlite3
from dataclasses import asdict, dataclass
from datetime import datetime, timezone
from pathlib import Path
from textwrap import dedent
from typing import Any, Dict, List, Tuple, Union

import pandas as pd
from crewai import Agent, Crew, Process, Task
from crewai_tools import tool
# from google.colab import userdata
from langchain.schema import AgentFinish
from langchain.schema.output import LLMResult
from langchain_community.tools.sql_database.tool import (
    InfoSQLDatabaseTool,
    ListSQLDatabaseTool,
    QuerySQLCheckerTool,
    QuerySQLDataBaseTool,
)
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_core.callbacks.base import BaseCallbackHandler
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq


In [7]:
df = pd.read_excel(r"C:\Users\HP\Documents\POC_X\Tasks 1.xlsx")
df.head()

,Task ID,Task Name,Owner,Priority,Start Date,Due Date,Created Time,Status,Is Overdue,Completion Percentage,...,Owner IDs,Task Delay Time,Task Completion Mode,Actual Time Taken,Time Spent So Far,Duration_1,Duration Unit,Rate Per Hour,Clarity Level,QC Owner
0,749813000020764032,Create Partial Invoice,Shavi Thakur,NaN,2024-03-15 09:30:00,2024-03-15 19:00:00,2024-03-15 18:26:00,To Do,Yes,0.0,...,773081417,3.0,Late,4.0,4,1.00,days,NaN,Clear,Mohammed Yusha
1,749813000020764032,CEEW -Daily Team Catchup,Stuti S. Kulshrestha,NaN,2024-03-15 09:30:00,2024-03-15 19:00:00,2024-03-15 09:41:00,In Progress,Yes,0.1,...,808004288,3.0,Late,4.0,4,1.00,days,NaN,Clear,Stuti S. Kulshrestha
2,749813000020761984,Auto populate Business Description and References,Pavan S,NaN,2024-03-16 14:15:00,2024-03-16 18:30:00,2024-03-16 14:11:00,To Do,Yes,0.0,...,845599619,2.0,Late,3.0,3,4.25,hrs,NaN,Not Clear,NaN
3,749813000020761984,OEM and Prod Category Reports,Achyuth T M,High,2024-03-15 09:30:00,2024-03-22 19:00:00,2024-03-15 10:27:00,In Progress,No,0.0,...,708853094,-4.0,On Time,4.0,4,6.00,days,NaN,NaN,NaN
4,749813000020760960,Map App status with Zoho,Vikram Pai V,NaN,2024-03-15 09:30:00,2024-03-15 19:00:00,2024-03-15 13:24:00,To Do,Yes,0.0,...,845596686,3.0,Late,4.0,4,1.00,days,NaN,NaN,NaN


In [12]:
import sqlite3

In [13]:
connection = sqlite3.connect("tasks.db")
df.to_sql(name="tasks", con=connection)

3302

In [17]:
db = SQLDatabase.from_uri("sqlite:///tasks.db")

In [22]:
query="""
SELECT COUNT(*) AS Abandoned_Task_Count 
FROM tasks 
WHERE Status = 'Terminated/Cancelled';"""

query1="""SELECT COUNT(*) FROM tasks WHERE Status IN ('In Progress', 'Development', 'Development Complete', 'Ready for QC', 'Discovery Phase', 'QA level 1 Completed', 'QA level 2 in progress');"""

In [ ]:
query2="""
SELECT * FROM tasks
WHERE "Task Delay Time" > 30;
"""

In [52]:
query2="""
SELECT * FROM tasks WHERE "Task Delay Time" > 30
"""

In [93]:
query3 = "SELECT * FROM tasks WHERE Owner LIKE '%Kiran%'"

In [99]:
query4="""
SELECT * FROM tasks WHERE Owner IDs LIKE '%,%'
"""

In [90]:
df.columns

Index(['Task ID', 'Task Name', 'Owner', 'Priority', 'Start Date', 'Due Date',
       'Created Time', 'Status', 'Is Overdue', 'Completion Percentage',
       'Project Name', 'Milestone Name', 'Project ID', 'Milestone ID',
       'Parent Task ID', 'Created By ID', 'Owner IDs', 'Task Delay Time',
       'Task Completion Mode', 'Actual Time Taken', 'Time Spent So Far',
       'Duration_1', 'Duration Unit', 'Rate Per Hour', 'Clarity Level',
       'QC Owner'],
      dtype='object')

In [91]:
# @tool("tables_schema")
# def tables_schema(tables: str) -> str:
#     """
#     Input is a comma-separated list of tables, output is the schema and sample rows
#     for those tables. Be sure that the tables actually exist by calling `list_tables` first!
#     Example Input: table1, table2, table3
#     """
#     tool = InfoSQLDatabaseTool(db=db)
#     return tool.invoke(tables)

# print(tables_schema.run("tasks"))

In [110]:
@tool("execute_sql")
def execute_sql(sql_query: str) -> str:
    """Execute a SQL query against the database. Returns the result"""
    return QuerySQLDataBaseTool(db=db).invoke(sql_query)

execute_sql.run("""SELECT * FROM tasks WHERE Owner LIKE '%Darhani%'""")

Using Tool: execute_sql


''

In [107]:
df['Task Delay Time']

0        3.0
1        3.0
2        2.0
3       -4.0
4        3.0
        ... 
3297     0.0
3298    80.0
3299     6.0
3300    76.0
3301    11.0
Name: Task Delay Time, Length: 3302, dtype: float64